In [40]:
import pandas as pd
from sklearn.impute import KNNImputer

In [41]:
df = pd.read_csv('telecom_churn_data.csv')

In [42]:
df.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.0,0.0,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.0,0.0,0.00,0.00


In [43]:
df.shape

(99999, 226)

In [44]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 226 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mobile_number             99999 non-null  int64  
 1   circle_id                 99999 non-null  int64  
 2   loc_og_t2o_mou            98981 non-null  float64
 3   std_og_t2o_mou            98981 non-null  float64
 4   loc_ic_t2o_mou            98981 non-null  float64
 5   last_date_of_month_6      99999 non-null  object 
 6   last_date_of_month_7      99398 non-null  object 
 7   last_date_of_month_8      98899 non-null  object 
 8   last_date_of_month_9      98340 non-null  object 
 9   arpu_6                    99999 non-null  float64
 10  arpu_7                    99999 non-null  float64
 11  arpu_8                    99999 non-null  float64
 12  arpu_9                    99999 non-null  float64
 13  onnet_mou_6               96062 non-null  float64
 14  onnet

## Data Cleaning

In [45]:
# function to sort missing percentages
def sortMissing(data, size):
    missing_df = round((data.isnull().sum() * 100/ len(data)),2).sort_values(ascending=False)
    print(missing_df.head(size))

In [46]:
# get the percentage of missing values
sortMissing(df, 50)

count_rech_2g_6             74.85
date_of_last_rech_data_6    74.85
count_rech_3g_6             74.85
av_rech_amt_data_6          74.85
max_rech_data_6             74.85
total_rech_data_6           74.85
arpu_3g_6                   74.85
arpu_2g_6                   74.85
night_pck_user_6            74.85
fb_user_6                   74.85
arpu_3g_7                   74.43
count_rech_2g_7             74.43
fb_user_7                   74.43
count_rech_3g_7             74.43
arpu_2g_7                   74.43
av_rech_amt_data_7          74.43
max_rech_data_7             74.43
night_pck_user_7            74.43
total_rech_data_7           74.43
date_of_last_rech_data_7    74.43
night_pck_user_9            74.08
date_of_last_rech_data_9    74.08
fb_user_9                   74.08
arpu_2g_9                   74.08
max_rech_data_9             74.08
arpu_3g_9                   74.08
total_rech_data_9           74.08
av_rech_amt_data_9          74.08
count_rech_3g_9             74.08
count_rech_2g_

In [47]:
# dropping all columns having high percentages of missing values >70%
for column in df:
    col = df[column]
    if col.isnull().sum() * 100 / len(df) > 70.0:
        df.drop([column], axis=1, inplace=True)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 186 entries, mobile_number to sep_vbc_3g
dtypes: float64(143), int64(35), object(8)
memory usage: 141.9+ MB


In [49]:
sortMissing(df, 10)

loc_ic_t2m_mou_9    7.75
std_og_t2t_mou_9    7.75
std_ic_t2f_mou_9    7.75
std_ic_t2t_mou_9    7.75
roam_og_mou_9       7.75
loc_ic_mou_9        7.75
loc_og_t2t_mou_9    7.75
loc_ic_t2f_mou_9    7.75
roam_ic_mou_9       7.75
ic_others_9         7.75
dtype: float64


### Dropping columns having single value

In [50]:
df.columns[df.nunique() <= 1].shape

(16,)

In [51]:
for column in df.columns[df.nunique() <= 1]:
    df.drop([column], axis=1, inplace=True)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 170 entries, mobile_number to sep_vbc_3g
dtypes: float64(132), int64(34), object(4)
memory usage: 129.7+ MB


### Filter High Value customers

In [53]:
avg_rech_amt = (df.total_rech_amt_6 + df.total_rech_amt_7) / 2
df['avg_rech_amt'] = avg_rech_amt
df['avg_rech_amt'].quantile(0.7)

368.5

In [54]:
df_high_val = df[df['avg_rech_amt'] >= df['avg_rech_amt'].quantile(0.7)]

In [55]:
df_high_val.shape

(30011, 171)

In [56]:
df_high_val.head()

,mobile_number,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,onnet_mou_7,onnet_mou_8,onnet_mou_9,offnet_mou_6,...,sachet_3g_6,sachet_3g_7,sachet_3g_8,sachet_3g_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,avg_rech_amt
7,7000701601,1069.180,1349.850,3171.480,500.000,57.84,54.68,52.29,NaN,453.43,...,0,0,0,0,802,57.74,19.38,18.74,0.0,1185.0
8,7001524846,378.721,492.223,137.362,166.787,413.69,351.03,35.08,33.46,94.66,...,0,0,0,0,315,21.03,910.65,122.16,0.0,519.0
13,7002191713,492.846,205.671,593.260,322.732,501.76,108.39,534.24,244.81,413.31,...,0,0,0,0,2607,0.00,0.00,0.00,0.0,380.0
16,7000875565,430.975,299.869,187.894,206.490,50.51,74.01,70.61,31.34,296.29,...,0,0,0,0,511,0.00,2.45,21.89,0.0,459.0
17,7000187447,690.008,18.980,25.499,257.583,1185.91,9.28,7.79,558.51,61.64,...,0,0,0,0,667,0.00,0.00,0.00,0.0,408.0


## Derive Churn

In [57]:
df_high_val.filter(regex=("._9")).columns

Index(['arpu_9', 'onnet_mou_9', 'offnet_mou_9', 'roam_ic_mou_9',
       'roam_og_mou_9', 'loc_og_t2t_mou_9', 'loc_og_t2m_mou_9',
       'loc_og_t2f_mou_9', 'loc_og_t2c_mou_9', 'loc_og_mou_9',
       'std_og_t2t_mou_9', 'std_og_t2m_mou_9', 'std_og_t2f_mou_9',
       'std_og_mou_9', 'isd_og_mou_9', 'spl_og_mou_9', 'og_others_9',
       'total_og_mou_9', 'loc_ic_t2t_mou_9', 'loc_ic_t2m_mou_9',
       'loc_ic_t2f_mou_9', 'loc_ic_mou_9', 'std_ic_t2t_mou_9',
       'std_ic_t2m_mou_9', 'std_ic_t2f_mou_9', 'std_ic_mou_9',
       'total_ic_mou_9', 'spl_ic_mou_9', 'isd_ic_mou_9', 'ic_others_9',
       'total_rech_num_9', 'total_rech_amt_9', 'max_rech_amt_9',
       'date_of_last_rech_9', 'last_day_rch_amt_9', 'vol_2g_mb_9',
       'vol_3g_mb_9', 'monthly_2g_9', 'sachet_2g_9', 'monthly_3g_9',
       'sachet_3g_9'],
      dtype='object')

In [58]:
df_high_val.total_og_mou_9.value_counts()

0.00       3249
0.21         20
1.01          7
0.61          7
5.93          6
           ... 
345.84        1
97.94         1
1391.18       1
248.33        1
1281.78       1
Name: total_og_mou_9, Length: 22615, dtype: int64

In [59]:
# total_og_mou_9, total_ic_mou_9, monthly_2g_9, monthly_3g_9
df_high_val['churn'] = (df_high_val['total_og_mou_9'] < 1.0) &  (df_high_val['total_ic_mou_9'] < 1.0) & (df_high_val['monthly_2g_9'] < 1.0) & (df_high_val['monthly_3g_9'] < 1.0)
df_high_val['churn'].value_counts

<ipython-input-59-a659f56c4925>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_high_val['churn'] = (df_high_val['total_og_mou_9'] < 1.0) &  (df_high_val['total_ic_mou_9'] < 1.0) & (df_high_val['monthly_2g_9'] < 1.0) & (df_high_val['monthly_3g_9'] < 1.0)


<bound method IndexOpsMixin.value_counts of 7         True
8        False
13       False
16       False
17       False
         ...  
99970    False
99974    False
99986    False
99988    False
99997    False
Name: churn, Length: 30011, dtype: bool>

In [60]:
df_high_val.head()

,mobile_number,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,onnet_mou_7,onnet_mou_8,onnet_mou_9,offnet_mou_6,...,sachet_3g_7,sachet_3g_8,sachet_3g_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,avg_rech_amt,churn
7,7000701601,1069.180,1349.850,3171.480,500.000,57.84,54.68,52.29,NaN,453.43,...,0,0,0,802,57.74,19.38,18.74,0.0,1185.0,True
8,7001524846,378.721,492.223,137.362,166.787,413.69,351.03,35.08,33.46,94.66,...,0,0,0,315,21.03,910.65,122.16,0.0,519.0,False
13,7002191713,492.846,205.671,593.260,322.732,501.76,108.39,534.24,244.81,413.31,...,0,0,0,2607,0.00,0.00,0.00,0.0,380.0,False
16,7000875565,430.975,299.869,187.894,206.490,50.51,74.01,70.61,31.34,296.29,...,0,0,0,511,0.00,2.45,21.89,0.0,459.0,False
17,7000187447,690.008,18.980,25.499,257.583,1185.91,9.28,7.79,558.51,61.64,...,0,0,0,667,0.00,0.00,0.00,0.0,408.0,False


In [61]:

df_high_val['churn'] = df_high_val['churn'].astype(int)

<ipython-input-61-4ef39163c142>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_high_val['churn'] = df_high_val['churn'].astype(int)


In [62]:
df_high_val['churn'].value_counts()

0    27302
1     2709
Name: churn, dtype: int64

In [63]:
### Checking the Churn Rate
churn = (sum(df_high_val['churn'])/len(df_high_val['churn'].index))*100
churn

9.02669021358835

In [64]:
df_high_val[['total_og_mou_9', 'total_ic_mou_9', 'monthly_2g_9', 'monthly_3g_9']].head()

,total_og_mou_9,total_ic_mou_9,monthly_2g_9,monthly_3g_9
7,0.00,0.00,0,0
8,142.18,34.91,0,0
13,446.09,631.86,0,0
16,255.74,334.56,0,0
17,631.08,87.13,0,0


### Drop all columns for Month 9

In [65]:
df_high_val.drop(df_high_val.filter(regex=("._9")).columns, inplace=True, axis=1)

/Users/sumitarora/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [66]:
df_high_val.shape

(30011, 131)

In [67]:
df_high_val.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30011 entries, 7 to 99997
Data columns (total 131 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   mobile_number        30011 non-null  int64  
 1   arpu_6               30011 non-null  float64
 2   arpu_7               30011 non-null  float64
 3   arpu_8               30011 non-null  float64
 4   onnet_mou_6          29695 non-null  float64
 5   onnet_mou_7          29708 non-null  float64
 6   onnet_mou_8          29073 non-null  float64
 7   offnet_mou_6         29695 non-null  float64
 8   offnet_mou_7         29708 non-null  float64
 9   offnet_mou_8         29073 non-null  float64
 10  roam_ic_mou_6        29695 non-null  float64
 11  roam_ic_mou_7        29708 non-null  float64
 12  roam_ic_mou_8        29073 non-null  float64
 13  roam_og_mou_6        29695 non-null  float64
 14  roam_og_mou_7        29708 non-null  float64
 15  roam_og_mou_8        29073 non-null

In [68]:
df_high_val.shape

(30011, 131)

In [69]:
# removing mobile number
df_high_val.drop(['mobile_number', 'date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8'], inplace=True, axis=1)
df_high_val.reset_index(inplace=True)

## Data Preparation

In [74]:
# let's check the distribution of churn column
sortMissing(df_high_val, 50)

loc_ic_t2f_mou_8    3.13
std_og_t2f_mou_8    3.13
spl_ic_mou_8        3.13
ic_others_8         3.13
std_og_t2m_mou_8    3.13
std_og_t2t_mou_8    3.13
std_ic_mou_8        3.13
loc_og_mou_8        3.13
std_ic_t2f_mou_8    3.13
loc_og_t2c_mou_8    3.13
std_ic_t2m_mou_8    3.13
loc_og_t2f_mou_8    3.13
std_ic_t2t_mou_8    3.13
loc_og_t2m_mou_8    3.13
loc_ic_mou_8        3.13
loc_og_t2t_mou_8    3.13
isd_ic_mou_8        3.13
og_others_8         3.13
loc_ic_t2m_mou_8    3.13
isd_og_mou_8        3.13
roam_ic_mou_8       3.13
loc_ic_t2t_mou_8    3.13
spl_og_mou_8        3.13
offnet_mou_8        3.13
roam_og_mou_8       3.13
std_og_mou_8        3.13
onnet_mou_8         3.13
loc_ic_t2f_mou_6    1.05
spl_ic_mou_6        1.05
spl_og_mou_6        1.05
std_og_t2f_mou_6    1.05
og_others_6         1.05
std_ic_t2f_mou_6    1.05
std_ic_mou_6        1.05
std_og_mou_6        1.05
std_ic_t2m_mou_6    1.05
loc_ic_t2t_mou_6    1.05
std_ic_t2t_mou_6    1.05
loc_ic_t2m_mou_6    1.05
isd_og_mou_6        1.05


In [77]:
# imputing missing values with KNN fancy imputer
imputer = KNNImputer(n_neighbors=2)

In [78]:
df_high_val_imputed = pd.DataFrame(imputer.fit_transform(df_high_val),columns = df_high_val.columns)

In [79]:
df_high_val_imputed.head()

,index,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,...,sachet_3g_6,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,avg_rech_amt,churn
0,7.0,1069.180,1349.850,3171.480,57.84,54.68,52.29,453.43,567.16,325.91,...,0.0,0.0,0.0,802.0,57.74,19.38,18.74,0.0,1185.0,1.0
1,8.0,378.721,492.223,137.362,413.69,351.03,35.08,94.66,80.63,136.48,...,0.0,0.0,0.0,315.0,21.03,910.65,122.16,0.0,519.0,0.0
2,13.0,492.846,205.671,593.260,501.76,108.39,534.24,413.31,119.28,482.46,...,0.0,0.0,0.0,2607.0,0.00,0.00,0.00,0.0,380.0,0.0
3,16.0,430.975,299.869,187.894,50.51,74.01,70.61,296.29,229.74,162.76,...,0.0,0.0,0.0,511.0,0.00,2.45,21.89,0.0,459.0,0.0
4,17.0,690.008,18.980,25.499,1185.91,9.28,7.79,61.64,0.00,5.54,...,0.0,0.0,0.0,667.0,0.00,0.00,0.00,0.0,408.0,0.0


In [81]:
sortMissing(df_high_val_imputed, 10)

churn               0.0
avg_rech_amt        0.0
std_og_t2t_mou_8    0.0
std_og_t2m_mou_6    0.0
std_og_t2m_mou_7    0.0
std_og_t2m_mou_8    0.0
std_og_t2f_mou_6    0.0
std_og_t2f_mou_7    0.0
std_og_t2f_mou_8    0.0
std_og_mou_6        0.0
dtype: float64
